In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10216546920150213332
]


In [26]:
import numpy as np, os
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings(action='ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

print(tf.__version__)
print(keras.__version__)

#한글설정
import matplotlib.font_manager as fm

font_dirs = ['/usr/share/fonts/truetype/nanum', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)
    
# 한글 출력을 위해서 폰트 옵션을 설정합니다.
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지

sns.set(font="NanumBarunGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

2.5.0
2.5.0


In [27]:
import pandas as pd

In [13]:
aa = pd.read_csv('./data/lstm_feature_importance_fold_1.csv')
baseline = aa[aa['feature']=='BASELINE'].baseline_mae.values[0]
aa['baseline_mae'] = aa['baseline_mae'].fillna(baseline)

In [15]:
aa['diff'] = aa['mae'] - aa['baseline_mae']

In [18]:
aaa = aa.sort_values('diff', ascending=False)

In [22]:
aaa

,feature,baseline_mae,mae,diff
0,0,0.478122,0.479388,0.001266
1,63323026201,0.478122,0.478833,0.000711
2,51079000220,0.478122,0.478641,0.000519
3,472500360,0.478122,0.478635,0.000513
4,574705050,0.478122,0.478522,0.000400
...,...,...,...,...
4065,50902,0.478122,0.477465,-0.000657
4066,50912,0.478122,0.476552,-0.001570
4067,51006,0.478122,0.476292,-0.001830
4068,51277,0.478122,0.475999,-0.002123


In [39]:
per_list = aaa['feature'][1:11].to_list()
per_list

['63323026201',
 '51079000220',
 '472500360',
 '574705050',
 '55390000401',
 '51221',
 '51222',
 '182050789',
 '121065721',
 '37205036174']

In [40]:
COLS = list(pd.read_csv('/project/LSH/total_data_7727.csv')['ITEMID'].sort_values().unique())
#COLS에서의 index 추출 (COLS의 index가 곧 3차원데이터 feature의 index이기 때문)

per_list_index = [COLS.index(int(i)) for i in per_list]
per_list_index

[3832, 3045, 2287, 2388, 3456, 174, 175, 1803, 1589, 2877]

In [41]:
import random
random.seed(42)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 


path = '/project/LSH/'
X =  np.load(path + 'x_(7727,4068).npy')[:,per_list_index]
y = np.load(path + 'y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)

i = round(X.shape[0]*0.8)
X_train, y_train = X[idx[:i],:], y[idx[:i]]
X_test, y_test = X[idx[i:],:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

model = RandomForestClassifier()
model.fit(X_train, y_train)

pred = model.predict(X_test)

print(f'정확도 : {metrics.accuracy_score(y_test, pred)}')

정확도 : 0.6330097087378641


In [42]:
import random    
seed_num = 42
random.seed(seed_num)
path = '/project/LSH/'
X = np.load(path + 'x_(7727,10,4068).npy')[:,:,per_list_index]
y = np.load(path + 'y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)


i = round(X.shape[0]*0.8)
X_train, y_train = X[idx[:i],:,:], y[idx[:i]]
X_test, y_test = X[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'rf_top10_seed42-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True, overwrite=True)

early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 4s 47ms/step - loss: 0.6759 - acc: 0.5984 - val_loss: 0.6660 - val_acc: 0.6177

Epoch 00001: val_loss improved from inf to 0.66601, saving model to ./model/rf_top10_seed42-01-0.6660.hdf5
Epoch 2/500
37/37 [==============================] - 1s 37ms/step - loss: 0.6732 - acc: 0.6066 - val_loss: 0.6662 - val_acc: 0.6177

Epoch 00002: val_loss did not improve from 0.66601
Epoch 3/500
37/37 [==============================] - 1s 37ms/step - loss: 0.6740 - acc: 0.6083 - val_loss: 0.6655 - val_acc: 0.6177

Epoch 00003: val_loss improved from 0.66601 to 0.66545, saving model to ./model/rf_top10_seed42-03-0.6655.hdf5
Epoch 4/500
37/37 [==============================] - 1s 37ms/step - loss: 0.6709 - acc: 0.6098 - val_loss: 0.6651 - val_acc: 0.6177

Epoch 00004: val_loss improved from 0.66545 to 0.66508, saving model to ./model/rf_top10_seed42-04-0.6651.hdf5
Epoch 5/500
37/37 [==============================] - 1s 37ms/step - loss: 0.6703 - acc: 


Epoch 00037: val_loss did not improve from 0.62779
Epoch 38/500
37/37 [==============================] - 1s 37ms/step - loss: 0.6336 - acc: 0.6557 - val_loss: 0.6279 - val_acc: 0.6546

Epoch 00038: val_loss did not improve from 0.62779
Epoch 39/500
37/37 [==============================] - 1s 37ms/step - loss: 0.6354 - acc: 0.6544 - val_loss: 0.6283 - val_acc: 0.6559

Epoch 00039: val_loss did not improve from 0.62779
Epoch 40/500
37/37 [==============================] - 1s 38ms/step - loss: 0.6341 - acc: 0.6594 - val_loss: 0.6281 - val_acc: 0.6552

Epoch 00040: val_loss did not improve from 0.62779
Epoch 41/500
37/37 [==============================] - 1s 37ms/step - loss: 0.6349 - acc: 0.6540 - val_loss: 0.6280 - val_acc: 0.6552

Epoch 00041: val_loss did not improve from 0.62779
Epoch 42/500
37/37 [==============================] - 1s 37ms/step - loss: 0.6336 - acc: 0.6534 - val_loss: 0.6284 - val_acc: 0.6527

Epoch 00042: val_loss did not improve from 0.62779
Epoch 43/500
37/37 [===

In [43]:
from pathlib import Path
paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
best_model_path = str(paths)
best_model_path

'model/rf_top10_seed42-31-0.6278.hdf5'

In [44]:
best_model = keras.models.load_model(best_model_path)
result = 0
for seed in range(0, 50):
    random.seed(seed)

    X = np.load(path + 'x_(7727,10,4068).npy')[:,:,per_list_index]
    y = np.load(path + 'y_(7727,1).npy')
    
    idx = list(range(len(X)))
    random.shuffle(idx)
    i = round(X.shape[0]*0.8)
    X_train, y_train = X[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = X[idx[i:],:,:], y[idx[i:]]

    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    result += acc
    print(f'정확도 :{acc}, seed_num = {seed}')
print(f'평균 정확도 : {result/50}')

정확도 :0.6466019417475728, seed_num = 0
정확도 :0.6576051779935275, seed_num = 1
정확도 :0.6647249190938511, seed_num = 2
정확도 :0.6498381877022654, seed_num = 3
정확도 :0.6401294498381876, seed_num = 4
정확도 :0.6699029126213593, seed_num = 5
정확도 :0.6537216828478964, seed_num = 6
정확도 :0.6466019417475728, seed_num = 7
정확도 :0.6621359223300971, seed_num = 8
정확도 :0.6627831715210356, seed_num = 9
정확도 :0.6517799352750809, seed_num = 10
정확도 :0.6673139158576051, seed_num = 11
정확도 :0.6550161812297735, seed_num = 12
정확도 :0.6666666666666666, seed_num = 13
정확도 :0.6627831715210356, seed_num = 14
정확도 :0.6660194174757281, seed_num = 15
정확도 :0.6724919093851133, seed_num = 16
정확도 :0.6459546925566343, seed_num = 17
정확도 :0.6550161812297735, seed_num = 18
정확도 :0.6504854368932039, seed_num = 19
정확도 :0.6621359223300971, seed_num = 20
정확도 :0.6466019417475728, seed_num = 21
정확도 :0.6588996763754046, seed_num = 22
정확도 :0.6530744336569579, seed_num = 23
정확도 :0.6498381877022654, seed_num = 24
정확도 :0.6634304207119741, seed_num =